In [21]:
import logging

import numpy as np
import pandas as pd
import geopandas as gpd
import pyarrow.compute as pc
from matplotlib import pyplot as plt
import pyarrow.dataset as ds
from shapely.geometry import Point, Polygon, MultiPolygon
from scipy.spatial import KDTree

from src.datalake import Datalake

logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)
plt.style.use('bmh')


def initialize_figure(w: int = 14, h: int = 10) -> any:
    return plt.subplots(figsize=(w, h))

In [5]:
data = Datalake("../data")
state_geoms = data.query_states()

scope_states = ["California"]
scope_multipoly = state_geoms[state_geoms.name.isin(scope_states)].geometry.iloc[0]
stations = data.query_stations()
stations = stations[stations.geometry.within(scope_multipoly)]
stations["num_samples"] = 0

In [30]:
dataset = ds.dataset(data.datalake_root / data.paths["ghcnd_clean_daily"])
for i, (row_id, s) in enumerate(stations.iterrows()):
    n = dataset.filter(pc.field("station_id") == s.station_id).to_table().num_rows
    stations.loc[row_id, "num_samples"] = n
    print(i, n)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241


KeyboardInterrupt: 